In [ ]:
# Necessary to display the plots in the notebook
%matplotlib inline

import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
from IPython.display import display, display_html, display_markdown, HTML, Markdown as md
from matplotlib.lines import Line2D
import seaborn as sns
import math
import time
import pickle
from typing import Union
from joblib import dump

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder, label_binarize, normalize
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import learning_curve

In [ ]:
%%html
<style>
@import url('https://fonts.googleapis.com/css2?family=DM+Sans:ital,opsz,wght@0,9..40,100..1000;1,9..40,100..1000&display=swap');
div.text_cell {font-family : DM Sans, sans-serif !important;}
pre {font-family : DM Sans, sans-serif !important;}
</style>

# **Clasification Model Comparison Template** 

This Notebook compares the performance of different types of classification models on a dataset provided by the user. It is meant to assist in the model selection process, helping users discern the most suitible classification technique for their dataset. Throughout the template there are global variables that the user must configure which represent characteristics of the dataset. Unchanging characteristics like the name of the data file are represented with capitalized constant variables. Users should store data in the data directory.

## **Dataset Overview**
In this example, a customer segmentation dataset from Kaggle is used to demonstate the templates function. The goal is to classify customers into different segments based on their characteristics. The dataset contains customer information including demographic data and purchasing behavior. Our target variable is 'Segmentation', which categorizes customers into segments A, B, C, or D.

## **Load in a Dataset and Select Columns to Drop**

In [ ]:
"""
In the data_file_name variable, input the name of the file containing your dataset (excluding /data. Example input: data_file.csv).

In the columns_to_drop variable, input the name of columns to drop from you dataset (columns that dont contain any useful information for regression).

In the dataset_na_value_representations variable, input all representations of missing values in your dataset that may not be automatically
recognized as missing. Make sure no valid value in your dataset is included in this list.
"""
data_file_name: str = "classification_segmentation_data.csv" 
columns_to_drop: list[str] = ["ID"] 
dataset_na_value_representations = ['', 'NA', 'N/A', 'null', 'NULL', 'NaN', 'none', 'None', '-', '?']


def display_dataframe(df: pd.DataFrame, font_size: int = 20) -> None:
    """
    Displays the passed in DataFrame with the specified font size.

    Args:
        df (pd.DataFrame): The DataFrame to be displayed.
        font_size (int): The font size at which the DataFrame should be displayed.

    Returns:
        None
    """
    df_html = df.to_html()
    styled_html = f'<div style="font-size: {font_size}px;">{df_html}</div>'
    display_html(HTML(styled_html))
    

def display_text(text: str, font_size: int = 18, font_weight: str = 'normal') -> None:
    """
    Displays the passed in text with the specified font size and font weight.

    Args:
        text (str): The text to be displayed.
        font_size (int): The font size at which the text should be displayed.
        font_weight (str): The font weight (e.g., 'normal', 'bold', 'bolder', 'lighter', or numeric value from 100 to 900).

    Returns:
        None
    """
    styled_html = f'<div style="font-size: {font_size}px; font-weight: {font_weight};">{text}</div>'
    display_html(HTML(styled_html))
    

def load_data(file_name: str = data_file_name,
              dropped_columns = columns_to_drop,
              na_value_representations: list[str] = dataset_na_value_representations) -> pd.DataFrame:
    """
    Loads in user's input file as a pandas DataFrame and converts various representations of missing values to NaN.
    The file should be stored in the 'data' directory.
    
    Args:
        file_name (str): Name of file containing data for clustering
        dropped_columns (list[str]): List of columns to drop from the dataframe
        na_value_representations (list[str]): List of strings that represent missing values in the dataset
    Returns:
        df (pd.DataFrame): Dataframe of variable values for all data entries
    """
    # Automatically prepends 'data/' to the file name
    file_name = "data/" + file_name
    file_extension = file_name.split(".")[-1]

    if file_extension == "csv":
        df = pd.read_csv(file_name)
    elif file_extension in ["xls", "xlsx"]:
        df = pd.read_excel(file_name)
    elif file_extension == "json":
        df = pd.read_json(file_name)
    else:
        raise ValueError("Unsupported file format or misspelled file name. Please upload a CSV, Excel, or JSON file and ensure the file name is spelled correctly.")
    
    # Replaces input representations of missing values with np.nan
    df = df.replace(na_value_representations, np.nan)
    
    df = df.drop_duplicates()
    df = df.drop(columns = dropped_columns)
    
    return df


data_df: pd.DataFrame = load_data()
initial_number_of_entries: int = len(data_df)

variable_list: list[str] = list(data_df.columns)
number_of_variables: int = len(variable_list)

numerical_variables: list[str] = list(data_df.select_dtypes(include = np.number).columns)
categorical_variables: list[str] = list(data_df.select_dtypes(exclude = np.number).columns)

# Converts all categorical variables to the Catagorical data type so they are recognized as such by ydata_profiling
data_df[categorical_variables] = data_df[categorical_variables].astype("category")

# Initial ydata-profiling report of the dataset before missing values and outliers are handled
#initial_dataset_report = ProfileReport(data_df, title = "Dataset Profiling Report (Before Handling Outliers/Missing Values)", progress_bar = False, explorative = True)
#initial_dataset_report

### **Dataset Missing Values Information**
The code in the following cell identifies numerical and categorical variables with missing values then provides users with useful infotmation for handling missing data. If missing values are present, it creates a DataFrame of rows with missing values and calculates summary statistics (number of entries with missing values, percentage of entries with missing values, and number of remaining entries in the dataset if all rows with missing values are dropped). These statistics are displayed along with up to the first 5 entries containing missing values. The displayed dataframe only includes columns that have one or more missing value. If there are no missing values in the dataset, it displays a message indicating so.

In [ ]:
# Automatically selects columns in the dataset with missing values, differentiating between numerical and categorical columns
numerical_columns_with_missing_values: list[str] = data_df[numerical_variables].columns[data_df[numerical_variables].isnull().any()].tolist()
categorical_columns_with_missing_values: list[str] = data_df[categorical_variables].columns[data_df[categorical_variables].isnull().any()].tolist()
all_columns_with_missing_values: list[str] = numerical_columns_with_missing_values + categorical_columns_with_missing_values

if len(all_columns_with_missing_values) != 0:
    print()
    # Dataframe containing all entries with missing values. The columns of this dataframe each have at least one missing value.
    entries_with_missing_values_df: pd.DataFrame = data_df[all_columns_with_missing_values][data_df[all_columns_with_missing_values].isnull().any(axis = "columns")]
    number_of_entries_with_missing_values: int = len(entries_with_missing_values_df)
    percent_of_entries_with_missing_values: float = (number_of_entries_with_missing_values / initial_number_of_entries) * 100  
    
    display_text(f"Total Number of Entries: {initial_number_of_entries}")
    display_text(f"Total Number of Entreis with at Least One Missing Value: {number_of_entries_with_missing_values} ({percent_of_entries_with_missing_values:.2f}% of Entries)")
    display_text(f"Number of Entries if all Rows with Missing Values are Dropped: {initial_number_of_entries - number_of_entries_with_missing_values}")
    print()
    display_text("Up to First 5 Entries with Missing Values:")
    display_dataframe(entries_with_missing_values_df.head(), font_size = 16)
else:
    print()
    display_text("No Missing Values in Dataset")

## **Handle Missing Values**
Use the code in the following cell to drop or impute missing values in your dataset. If you chose to impute missing values, edit the imputute_missing_values function to suite your dataset (such as choosing which columns to impute and specifying between imputing with the mean, median, or mode of the column). If called without further specification, the provided imputute_missing_values function is applied to all columns with missing values, imputing missing numerical values with the median of the column and missing categorical values with the mode of the column. By default, the cell uses the drop_rows_with_missing_values function to drop all entries with one or more missing value. The drop_rows_with_missing_values and impute_missing_values functions can be used in conjunction to drop rows with missing values in columns that cannot be imputed and then impute the remaining missing values.

In [ ]:
def drop_rows_with_missing_values(df: pd.DataFrame = data_df,
                                  columns_to_check: list[str] = all_columns_with_missing_values) -> pd.DataFrame:
    """
    Makes a copy of the input DataFrame and drops rows that have one or more missing values in any of the columns specified by 
    the columns_to_check parameter (does not mutate the input DataFrame). Also prints the number of entries dropped and the
    resulting total number of entries.
    
    Args:
        df (pd.DataFrame): DataFrame containing loded in data
        columns_to_check (list[str]): List of columns to check for missing values
    Returns:
        dropna_df (pd.DataFrame): DataFrame with missing values dropped
    """
    
    original_number_of_entries = len(df)
    dropna_df = df.copy()
    
    dropna_df[columns_to_check] = dropna_df[columns_to_check].dropna()
    new_number_of_entries = len(dropna_df)
    number_of_entries_dropped = original_number_of_entries - new_number_of_entries
    
    display_text(f"drop_rows_with_missing_values Results: {number_of_entries_dropped} Entries Dropped") 
    display_text(f"New Number of Entries: {new_number_of_entries}")
    
    return dropna_df


def impute_missing_values(df: pd.DataFrame = data_df,
                          numerical_columns_to_impute: list[str] = numerical_columns_with_missing_values,
                          categorical_columns_to_impute: list[str] = categorical_columns_with_missing_values) -> pd.DataFrame:
    """
    Imputes missing values in the DataFrame with either the median value (for numerical variables) or the most frequent value
    (for categorical variables).
    
    Args:
        numerical_columns_to_impute (list[str]): List of the names of numerical columns with missing values to impute
        categorical_columns_to_impute (list[str]): List of the names of categorical columns with missing values to impute
    Returns:
        impute_df (pd.DataFrame): DataFrame with missing values imputed
    """
    impute_df = df.copy()
    
    # Here is where to configure the imputation strategy if need be
    numerical_imputer = SimpleImputer(strategy = "median")
    categorical_imputer = SimpleImputer(strategy = "most_frequent")
    
    impute_df[numerical_columns_to_impute] = numerical_imputer.fit_transform(impute_df[numerical_columns_to_impute])
    impute_df[categorical_columns_to_impute] = categorical_imputer.fit_transform(impute_df[categorical_columns_to_impute])
    
    display_text("Missing Values Successfully Imputed")
    
    return impute_df


"""
Specify how you would like to handle missing values in the dataset, setting a variable to hold your new dataset.
"""
data_df = drop_rows_with_missing_values()

## **Handle Outliers/Eronious Entries**
Use the visualize_outliers function to analyze numerical columns of your dataset for outliers/eronious inputs and optionally remove them. When removing entries with the visualize_outliers function, it is easiest to work with one column at a time.

In [ ]:
# Displayed to compare dataset before and after handling outliers/eronious entries
display_text("Numerical Variables Information", font_size = 20)
display_dataframe(data_df.describe(), 16) 

def visualize_outliers(df: pd.DataFrame = data_df,
                       numerical_columns_to_check: Union[list[str], str] = numerical_variables,
                       iqr_multiplier: float = 1.5,
                       remove: bool = False,
                       remove_option: str = 'both',
                       display: bool = True,) -> pd.DataFrame:
    """
    Creates a boxplot for each column of the input Dataframe in the numerical_columns_to_check parameter to help users visualize potential
    outliers in their dataset. Below this boxplot, the function prints the number of high and low outliers (determined by the IQR method) in the
    current column. The upper and lower bounds for outliers are denoted by red dotted lines. Points below the low bound red dotted line
    or above the high bound red dotted line are consideered outliers. Users can choose whether to drop outlier entries through the remove
    boolean parameter. You can change which points are considered outliers by changing the iqr_multiplier parameter.
    
    The lower and upper whiskers of the boxplot denote the 5th and 95th percentile of the current column's values respectively.

    This function can be used iteratively to handle outliers in different columns with varying sensitivity levels. It allows for
    selective removal of entries below/above the red dotted lines. The function can be run without displaying visualizations for efficiency.

    Args:
        df (pd.DataFrame): The input DataFrame containing the data to be analyzed.
        numerical_columns_to_check (Union[list[str], str], default = numerical_variables): List of the names of columns to check for outliers. The
            default argument is a list of all numerical columns in the input DataFrame.
        iqr_multiplier (float, default = 1.5): Multiplier for the IQR to define the outlier threshold. Higher values are more lenient, increasing
            the range of the upper and lower red dotted lines. Lower values are more strict, decreasing the range of the red dotted lines.
        remove (bool, default = False): If True, removes identified outliers from the DataFrame.
        remove_option (str, default = 'both'): Specifies which outliers to remove: 'both' removes all identified outliers, 'upper' only removes
            outliers greater than the upper bound (values past the upper red dotted line), and 'lower' only removes outliers less than the
            lower bound (values behind the lower red dotted line). This parameter has no effect if remove = False.
        display (bool, default = True): If True, displays boxplots for each variable. If false, only outlier statistics are printed.

    Returns:
        pd.DataFrame: The original DataFrame if remove = False, otherwise a new DataFrame with outliers removed.
    """
    # If a single column is passed in as a string, convert it to a list so the following for loop still works properly
    if type(numerical_columns_to_check) == str:
        numerical_columns_to_check = [numerical_columns_to_check]
        
    for col in numerical_columns_to_check:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (iqr * iqr_multiplier)
        upper_bound = q3 + (iqr * iqr_multiplier)
        
        # Only create plot if display is True
        if display:
            plt.figure(figsize = (10, 6))
            ax = sns.boxplot(x = df[col], whis = [5, 95])
            plt.title(f'Boxplot of {col}')
            
            # Add vertical red dotted lines for lower and upper bounds if within the plot's x-axis limits
            x_min, x_max = ax.get_xlim()
            if x_min <= lower_bound <= x_max:
                plt.axvline(lower_bound, color='red', linestyle='dotted', linewidth=1)
            if x_min <= upper_bound <= x_max:
                plt.axvline(upper_bound, color='red', linestyle='dotted', linewidth=1)
            
            # Create legend
            legend_lines = [Line2D([0], [0], color='red', linestyle='dotted', linewidth=1)]
            legend_labels = ['Lower/Upper Bound']
            plt.legend(legend_lines, legend_labels, loc='upper right')
            plt.show()
        
        lower_outlier_count = df[col][df[col] < lower_bound].count()
        upper_outlier_count = df[col][df[col] > upper_bound].count()
        
        display_text(f"{col}:", font_size = 18, font_weight = 'bold')
        display_text(f"- Lower Bound for Outliers: {lower_bound}", font_size = 16)
        display_text(f"- Upper Bound for Outliers: {upper_bound}", font_size = 16)
        display_text(f"- Number of Outliers Below Lower Bound: {lower_outlier_count}", font_size = 16)
        display_text(f"- Number of Outliers Above Upper Bound: {upper_outlier_count}", font_size = 16)
        print()
        
    # Removes outliers from the DataFrame if remove = True
    if remove:
        # Calculate indices of outliers
        lower_outlier_indices = df.index[df[col] < lower_bound].tolist()
        upper_outlier_indices = df.index[df[col] > upper_bound].tolist()
        outlier_indices_to_be_removed = set()
        
        # Add outlier indices that will be removed to outlier_indices_to_be_removed based on the remove_option parameter
        if remove_option == "both":
            outlier_indices_to_be_removed.update(lower_outlier_indices)
            outlier_indices_to_be_removed.update(upper_outlier_indices)
        elif remove_option == "lower":
            outlier_indices_to_be_removed.update(lower_outlier_indices)
        elif remove_option == "upper":
            outlier_indices_to_be_removed.update(upper_outlier_indices)
        else:
            raise ValueError("Invalid argument passed into remove_option parameter. Please use 'both', 'lower', or 'upper'.")
            
        removed_outliers_df = df.drop(index = outlier_indices_to_be_removed)
        display_text(f"Total Number of Outlier Entries Removed in {col}: {len(outlier_indices_to_be_removed)}", font_size = 18)
        print()
        return removed_outliers_df
    
    # Simply return the original DataFrame if remove = False
    return df


"""
Use the visualize_outliers function to identify and optionally remove outliers in the numerical columns of your dataset. To effectively
handle outliers, you may need to run this function multiple times with different sensitivity levels (iqr_multiplier) for different
columns. This means you may need to run the function multiple times with different columns_to_check parameters, handling each column
individually.
"""  
data_df = visualize_outliers(data_df, display = True)

## **Setup Preprocessing**

### **Dataset Preprocessing Information**
Use the information provided by the comparison profile report to analyze the result of your data cleaning and to help determine your preprocessing steps.

In [ ]:
dataset_report = ProfileReport(data_df, title = "Dataset Profiling Report (After Handling Outliers/Missing Values)", progress_bar = False, explorative = True)
post_cleaning_comparison_report = dataset_report.compare(initial_dataset_report)
post_cleaning_comparison_report

### **Preprocessing Steps**
Define your preprocessing steps in the code cell below. The provided steps address three common preprocessing transformations: scaling numerical variables, one-hot encoding nominal categorical variables, and ordianal encoding ordinal categorical variables. Customize these steps to fit the needs of your dataset. Preprocessing for your target variable and feature variables must be handled by seperate transformer variables.

The feature variable preprocessing steps will be passed into the Scikit Learn make_pipeline function when models are loaded in.

The target variable preprocessing steps will be applied directly to the y_train and y_test sets after the dataset is split.

In [ ]:
TARGET_COLUMN_NAME: str = "Segmentation"


numerical_features_to_scale: list[str] = list(set(numerical_variables) - set([TARGET_COLUMN_NAME]))
nominal_categorical_features_to_encode: list[str] = list(set(categorical_variables) - set([TARGET_COLUMN_NAME]))

# Order ordinal variable categories from lowest to highest 
ordianl_categories_ordered_dict: dict[str, list[str]] = {
    "Spending_Score": ['Low', 'Average', 'High']
}
ordianl_features_categories_to_encode: list[str] = list(ordianl_categories_ordered_dict.keys())
ordianl_features_categories_orders_lists: list[list[str]] = list(ordianl_categories_ordered_dict.values())

# Indicates that the first column of one-hot encoded variables should be dropped to avoid multicollinearity
onehot_drop_column = "first"


# Pass in a list of tuples containing a name for the transformer (decide a name, allows transformer parameters to be searched in grid search),
# the transformer object, and the columns to apply the transformer to
general_feature_preprocessor = ColumnTransformer(
    transformers = [
        ('numerical_scaler', StandardScaler(), numerical_features_to_scale),
        ('nominal_encoder', OneHotEncoder(drop = onehot_drop_column), nominal_categorical_features_to_encode),
        ("ordinal_encoder", OrdinalEncoder(categories = ordianl_features_categories_orders_lists), ordianl_features_categories_to_encode)
    ]
)

# Setup target preprocessor
nominal_target_preprocessor = LabelEncoder()

display_markdown(md(f"### **Scaled Numerical Features:** {numerical_features_to_scale}"))
display_markdown(md(f"### **Encoded Nominal Categorical Features:** {nominal_categorical_features_to_encode}"))
display_markdown(md(f"### **Encoded Ordinal Categorical Features (confirm that category orders were assigned to the correct ordinal categorical feature):**"))
for i in range(len(ordianl_features_categories_to_encode)):
    display_markdown(md(f"* #### **{ordianl_features_categories_to_encode[i]}:** {ordianl_features_categories_orders_lists[i]}"))

In [ ]:
"""
Input the name of the target variable column in the dataset
"""
target_column_name: str = "GradeClass"

"""
From here, you must create preprocessing steps specific  to your dataset. The following is a general preprocessing setup that can be
easily adapted to work on most datasets:
"""

"""
Input a list of the numerical features you would like to scale (defaultes to all numerical features in the dataset) and a list of
the nominal categorical features you would like to one-hot encode.
"""
numerical_features_to_scale: list[str] = ["Age", "StudyTimeWeekly", "Absences", ]
nominal_categorical_features_to_encode: list[str] = ["Ethnicity"]

"""
In the ordianl_categories_ordered_dict variable, input the order of ordinal categorical variable categories into a dictionary. The keys
of this dictionary should be names of ordinal categorical variables and the values should be lists of the categories in order from smallest
to largest.
"""
ordianl_categories_ordered_dict: dict[str, list[str]] = {
    "ParentalEducation": [0, 1, 2, 3, 4],
    "ParentalSupport": [0, 1, 2, 3, 4]
}

# Extracts the keys and values of the ordianl_categories_ordered_dict into separate lists
ordianl_feature_categories_to_encode: list[str] = list(ordianl_categories_ordered_dict.keys())
ordianl_feature_categories_orders_lists: list[list[str]] = list(ordianl_categories_ordered_dict.values())

# Indicates that the first column of one-hot encoded variables should be dropped to avoid multicollinearity
onehot_drop_column: str = "first"


# The argumnet for the transformers parameter of ColumnTransformer must be a a list of touples with three entries. Each of these touples
# represents a preprocessing step. The first entry of each touple is a name for the step. The second entry is the transformer object, and
# the final entry is a list of the columns the step should be applied to.
general_feature_preprocessor = ColumnTransformer(
    transformers = [
        ('numerical_scaler', StandardScaler(), numerical_features_to_scale),
        ('nominal_encoder', OneHotEncoder(drop = onehot_drop_column), nominal_categorical_features_to_encode),
        ("ordinal_encoder", OrdinalEncoder(categories = ordianl_feature_categories_orders_lists), ordianl_feature_categories_to_encode)
    ]
)

"""
Specify the preprocessor used on the target variable column.
"""
numerical_target_preprocessor = StandardScaler()

display_text(f"Scaled Numerical Variables: {numerical_features_to_scale}")
display_text(f"Encoded Nominal Categorical Variables: {nominal_categorical_features_to_encode}")
display_text(f"Encoded Ordinal Categorical Variables (confirm that category orders were assigned to the correct ordinal categorical variable):")
if len(ordianl_feature_categories_to_encode) != 0:
    for i in range(len(ordianl_feature_categories_to_encode)):
        display_markdown(md(f"* {ordianl_feature_categories_to_encode[i]}: {ordianl_feature_categories_orders_lists[i]}"))
else:
    display_text("None")

## **Load In Models and Set Hyperparameters**
Define the functions for loading each model. Use the make_pipeline function to combine the preprocessing steps with the model and set the hyperparameters for each model to be optimized by GridSearchCV.

The make_pipeline function changes the way you have to input values in each models param_grid. In order for GridSearchCV to work with the make_pipline function, model hyeperparameter keys must be prefixed with the model name followed by two underscores (ex: the hyperparameter for the Lasso Regression model is "lasso__alpha" instead of just "alpha").

In [ ]:
"""
Configure the models you would like to compare. Assign a variable for each model to a dictionary. The key of each dictionary should be a string containing
the name of the model. The value should be another dictionary with two string keys: "model" and "param_grid". The value of the "model" key should be a
pipeline (using the scikit-learn make_pipline function) that that combines your preprocessor with the model. The value of the "param_grid" key should be
a dictionary that defines the hyperparameters to be optimized for the model. In the "param_grid" dictionary, the keys should be model hyperparmeters and
the values should be list-like objects containing values to be tested for each hyperparameter by GridSearchCV. In order for GridSearchCV to recognize the
hyperparameters of a model that is part of a pipeline, the hyperparameter keys in the "param_grid" dictionary should be prefixed with the name of the
model followed by two underscores. For example, if you want to optimize the n_neighbors hyperparameter of a KNeighborsClassifier model, the key in the
"param_grid" dictionary for the alpha hyperparameter should be "kneighborsclassifier__n_neighbors". The prefixed model names are the same as the
scikit-learn model names in all lower case letters.
"""
logistic_model_data = {
    'Logistic Regression': {
        'model': make_pipeline(general_feature_preprocessor, LogisticRegression(max_iter=1000)),
        'param_grid': {
            'logisticregression__C': [0.1, 1, 10],
            'logisticregression__solver': ['liblinear', 'saga']
        }
    }
}

kneighbors_model_data = {
    'KNN': {
        'model': make_pipeline(general_feature_preprocessor, KNeighborsClassifier()),
        'param_grid': {
            'kneighborsclassifier__n_neighbors': [3, 5, 7],
            'kneighborsclassifier__weights': ['uniform', 'distance']
        }
    }
}

decision_tree_model_data = {
    'Decision Tree': {
        'model': make_pipeline(general_feature_preprocessor, DecisionTreeClassifier(random_state=42)),
        'param_grid': {
            'decisiontreeclassifier__max_depth': [None, 5, 10],
            'decisiontreeclassifier__min_samples_split': [2, 5, 10]
        }
    }
}

random_forest_model_data = {
    'Random Forest': {
        'model': make_pipeline(general_feature_preprocessor, RandomForestClassifier(random_state=42)),
        'param_grid': {
            'randomforestclassifier__n_estimators': [100, 200],
            'randomforestclassifier__max_depth': [None, 5, 10]
        }
    }
}

svc_model_data = {
        'SVC': {
            'model': make_pipeline(general_feature_preprocessor, SVC(probability=True, random_state=42)),
            'param_grid': {
                'svc__C': [0.1, 1, 10],
                'svc__kernel': ['linear', 'rbf']
            }
        }
    }


gaussian_nb_model_data = {
    'Naive Bayes': {
        'model': make_pipeline(general_feature_preprocessor, GaussianNB()),
        'param_grid': {}
    }
}


models = {
    **logistic_model_data,
    **kneighbors_model_data,
    **decision_tree_model_data,
    **random_forest_model_data,
    **svc_model_data,
    **gaussian_nb_model_data
}

## **Model Training and Evaluation**
Split the data into training and testing sets using the scikit-learn train_test_split function and transform the target variable using your target preprocessing steps. After this, the train_and_evaluate_models function will have access to all of it's necessary variables.

In [ ]:
NUM_DECIMAL_PLACES: int = 7 # Determines the number of decimal places to display for model evaluation metrics



X = data_df.drop(columns = [TARGET_COLUMN_NAME])
y = data_df[TARGET_COLUMN_NAME]

FEATURE_LIST = list(X.columns)
NUM_FEATURES: int = len(FEATURE_LIST)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = nominal_target_preprocessor.fit_transform(y_train)
y_test = nominal_target_preprocessor.fit_transform(y_test)

def train_and_evaluate_models(models_dict: dict[dict[str, dict]] = models,
                              X_train: pd.DataFrame = X_train,
                              X_test: pd.DataFrame = X_test,
                              y_train: np.ndarray = y_train,
                              y_test: np.ndarray = y_test) -> dict:
    """
    Optimizes the hyperparameters of, trains, and evaluates the performance of all passed in models on the training and testing data.
    Also saves information from the training process such as best model, predictions, and training/testing time.
    
    Args:
        models_dict (dict[dict[str, dict]]): Dictionary containing model names as keys and dictionaries containing the model object and hyperparameter grid as values
        X_train (pd.DataFrame): DataFrame containing feature variable values for training the models
        X_test (pd.DataFrame): DataFrame containing feature variable values for testing the models
        y_train (np.ndarray): 1D np.ndarray containing target variable values for training the models
        y_test (np.ndarray): 1D np.ndarray containing target variable values for testing the models
    Returns:
        model_results (dict): Dictionary that has model names as its keys. The value for these keys are dictionaries containing the trained model object,
        model predictions on the testing data, and other data from the training process such as accuracy, precision, recall, and F1 scores.
    """
    
    model_results = {}
    
    for model_name, model_data in models_dict.items():
        display_markdown(md(f"### **Training and evaluating: {model_name}**"))
        
        model = model_data['model']
        param_grid = model_data['param_grid']
        
        stratifiedkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=stratifiedkf, scoring='accuracy')
        
        tune_train_start_time = time.time()
        grid_search.fit(X_train, y_train)
        tune_train_end_time = time.time()
        tune_train_time = tune_train_end_time - tune_train_start_time
        display_markdown(md(f"* #### **Hyperparameter Tuning and Model Training Time:** {tune_train_time:.2f} seconds"))
        
        best_model = grid_search.best_estimator_
        
        y_train_predictions = best_model.predict(X_train)
        y_test_predictions = best_model.predict(X_test)
        y_test_prediction_probabilities = best_model.predict_proba(X_test)
        
        accuracy = accuracy_score(y_test, y_test_predictions)
        precision = precision_score(y_test, y_test_predictions, average='weighted')
        recall = recall_score(y_test, y_test_predictions, average='weighted')
        f1 = f1_score(y_test, y_test_predictions, average='weighted')
        
        display_markdown(md(f"* #### **Test Accuracy:** {accuracy:.{NUM_DECIMAL_PLACES}f}"))
        display_markdown(md(f"* #### **Test Precision:** {precision:.{NUM_DECIMAL_PLACES}f}"))
        display_markdown(md(f"* #### **Test Recall:** {recall:.{NUM_DECIMAL_PLACES}f}"))
        display_markdown(md(f"* #### **Test F1 Score:** {f1:.{NUM_DECIMAL_PLACES}f}"))
        
        model_results[model_name] = {
            'best_model': best_model,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            "cv_results": grid_search.cv_results_,
            "tune_train_time": tune_train_time,
            "y_train_predictions": y_train_predictions,
            "y_test_predictions": y_test_predictions,
            "y_test_prediction_probabilities": y_test_prediction_probabilities,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1
        }
    
    return model_results

model_results = train_and_evaluate_models()

### **Comparative Model Performance**

In [ ]:
def plot_comparative_model_performance(model_results: dict[str, dict[str, object]]):
    """
    Creates a grouped bar chart to compare performance metrics across different classification models.

    Args:
    model_results (Dict[str, Dict[str, Any]]): Dictionary containing model names as keys and their results as values.
                                               Each model's results should include 'f1', 'accuracy', 'precision', and 'recall'.

    Returns:
    None
    """
    # Extract model names and metrics
    models = list(model_results.keys())

    # Prepare data for plotting
    f1_scores = [model_results[model]['f1'] for model in models]
    accuracy_scores = [model_results[model]['accuracy'] for model in models]
    precision_scores = [model_results[model]['precision'] for model in models]
    recall_scores = [model_results[model]['recall'] for model in models]

    # Set up the plot
    fig, axs = plt.subplots(2, 2, figsize=(20, 20))
    fig.suptitle('Comparative Model Performance (Higher is Better for All)', fontsize=24)
    
    # Add separation between the main title and subplots
    plt.subplots_adjust(top=0.95, hspace=0.3, wspace=0.3)

    # Function to plot metric and set y-axis limits
    def plot_metric(ax, scores, title, color):
        x = np.arange(len(models))  # the label locations
        ax.bar(x, scores, color=color)
        ax.set_ylabel(title)
        ax.set_title(f'{title} Scores by Model', fontweight='bold')
        
        # Set tick locations and labels
        ax.set_xticks(x)
        ax.set_xticklabels(models, rotation=45, ha='right')
        
        # Set y-axis limits relative to the scores
        min_score = min(scores)
        max_score = max(scores)
        range_score = max_score - min_score
        ax.set_ylim(max(0, min_score - 0.1 * range_score), min(1, max_score + 0.1 * range_score))
        
        # Add value labels on the bars
        for i, v in enumerate(scores):
            ax.text(i, v, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

    # Plot F1 scores
    plot_metric(axs[0, 0], f1_scores, 'F1 Score', 'skyblue')
    
    # Plot Accuracy scores
    plot_metric(axs[0, 1], accuracy_scores, 'Accuracy', 'lightgreen')
    
    # Plot Precision scores
    plot_metric(axs[1, 0], precision_scores, 'Precision', 'salmon')
    
    # Plot Recall scores
    plot_metric(axs[1, 1], recall_scores, 'Recall', 'gold')

    # Add grid to all subplots
    for ax in axs.flat:
        ax.grid(True, linestyle=':', alpha=0.6)

    plt.tight_layout(rect=[0, 0, 1, 0.975])  # Add space at the top for the main title
    plt.show()


plot_comparative_model_performance(model_results)

### **Results Summary**

In [ ]:
def summarize_results(results: dict = model_results):
    """
    Displays the performance metrics of all models trained and evaluated in the train_and_evaluate_models function in a DataFrame
    and returns the name of the best model based on accuracy score. The model with the highest accuracy score is considered the best model.

    Args:
        results (dict): Dictionary containing model names as keys and dictionaries containing the trained model object,
                        predictions on the testing data, accuracy, precision, recall, F1 score, and other model performance data as values
    Returns:
        best_performing_model (str): Name of the best model based on accuracy score
    """

    summary_df = pd.DataFrame({
        'Model': results.keys(),
        'F1 Score': [result['f1'] for result in results.values()],
        'Accuracy': [result['accuracy'] for result in results.values()],
        'Precision': [result['precision'] for result in results.values()],
        'Recall': [result['recall'] for result in results.values()]
    })

    display(md("## Model Performance Summary"))
    display_dataframe(summary_df)
    print()

    best_performing_model = summary_df.loc[summary_df['F1 Score'].idxmax(), 'Model']
    display(md(f"## **Best Performing Model: {best_performing_model}** (based on F1 score, the higher the better)"))
    display(md(f"* ### F1 Score: {results[best_performing_model]['f1']:.{NUM_DECIMAL_PLACES}f}"))
    display(md(f"* ### Accuracy Score: {results[best_performing_model]['accuracy']:.{NUM_DECIMAL_PLACES}f}"))
    display(md(f"* ### Precision Score: {results[best_performing_model]['precision']:.{NUM_DECIMAL_PLACES}f}"))
    display(md(f"* ### Recall Score: {results[best_performing_model]['recall']:.{NUM_DECIMAL_PLACES}f}"))

    return best_performing_model

best_model = summarize_results()

### **Comparative Model Performance Visualizations**

In [ ]:
target_classes = nominal_target_preprocessor.classes_

def plot_confusion_matrices_comparison(model_results: dict[str, dict], y_test: np.ndarray, target_classes: np.ndarray):
    """
    Plots confusion matrices for multiple classification models in a grid layout.
    
    Args:
    model_results (dict[str, dict]): Dictionary of model results, where each key is a model name
                                     and each value is a dictionary containing model predictions
    y_test (np.ndarray): Actual target values
    target_classes (np.ndarray): Array of target class names
    
    Returns:
    None
    """
    num_models = len(model_results)
    num_rows = math.ceil(num_models / 2)
    fig, axes = plt.subplots(num_rows, 2, figsize=(20, 10*num_rows))
    
    fig.suptitle('Comparison of Confusion Matrices', fontsize=24, y=1.02)
    
    axes_flat = axes.flatten() if num_models > 1 else [axes]
    
    for ax, (model_name, results) in zip(axes_flat, model_results.items()):
        y_pred = results['y_test_predictions']
        cm = confusion_matrix(y_test, y_pred)
        
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_classes)
        disp.plot(ax=ax, cmap="Blues", values_format='d')
        
        ax.set_title(f'{model_name} Confusion Matrix', fontsize=18, fontweight='bold', pad=20)
        ax.tick_params(axis='both', which='major', labelsize=12)
        
        # Calculate and display accuracy
        accuracy = accuracy_score(y_test, y_pred)
        ax.text(0.5, -0.1, f'Accuracy: {accuracy:.3f}', 
                horizontalalignment='center', verticalalignment='center', 
                transform=ax.transAxes, fontsize=14, fontweight='bold')
    
    for ax in axes_flat[num_models:]:
        ax.set_visible(False)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.95, hspace=0.3, wspace=0.3)
    plt.show()
    

def plot_roc_curves_comparison(model_results: dict[str, dict], y_test: np.ndarray, target_classes: np.ndarray):
    """
    Plots ROC curves for multiple classification models in a grid layout.
    
    Args:
    model_results (dict[str, dict]): Dictionary of model results, where each key is a model name
                                     and each value is a dictionary containing model predictions
    y_test (np.ndarray): Actual target values
    target_classes (np.ndarray): Array of target class names
    
    Returns:
    None
    """
    num_models = len(model_results)
    num_rows = math.ceil(num_models / 2)
    fig, axes = plt.subplots(num_rows, 2, figsize=(20, 10*num_rows))
    
    fig.suptitle('Comparison of ROC Curves', fontsize=24, y=1.02)
    
    axes_flat = axes.flatten() if num_models > 1 else [axes]
    
    n_classes = len(target_classes)
    y_bin = label_binarize(y_test, classes=range(n_classes))
    
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
    
    for ax, (model_name, results) in zip(axes_flat, model_results.items()):
        y_prediction_probabilities = results['y_test_prediction_probabilities']
        
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_prediction_probabilities[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            
            ax.plot(fpr[i], tpr[i], color=colors[i % len(colors)], lw=2,
                    label=f'ROC curve of class {target_classes[i]} (AUC = {roc_auc[i]:.2f})')
        
        ax.plot([0, 1], [0, 1], 'k--', lw=2)
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title(f'{model_name} ROC Curve', fontsize=18, fontweight='bold', pad=20)
        ax.legend(loc="lower right", fontsize=10)
        ax.tick_params(axis='both', which='major', labelsize=12)
    
    for ax in axes_flat[num_models:]:
        ax.set_visible(False)
    
    plt.tight_layout(rec)
    plt.subplots_adjust(top=0.95, hspace=0.3, wspace=0.3)
    plt.show()


def plot_precision_recall_curves_comparison(model_results: dict[str, dict], y_test: np.ndarray, target_classes: np.ndarray):
    """
    Plots Precision-Recall curves for multiple classification models in a grid layout.
    
    Args:
    model_results (dict[str, dict]): Dictionary of model results, where each key is a model name
                                     and each value is a dictionary containing model predictions
    y_test (np.ndarray): Actual target values
    target_classes (np.ndarray): Array of target class names
    
    Returns:
    None
    """
    num_models = len(model_results)
    num_rows = math.ceil(num_models / 2)
    fig, axes = plt.subplots(num_rows, 2, figsize=(20, 10*num_rows))
    
    fig.suptitle('Comparison of Precision-Recall Curves', fontsize=24, y=1.02)
    
    axes_flat = axes.flatten() if num_models > 1 else [axes]
    
    n_classes = len(target_classes)
    y_bin = label_binarize(y_test, classes=range(n_classes))
    
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
    
    for ax, (model_name, results) in zip(axes_flat, model_results.items()):
        y_prediction_probabilities = results['y_test_prediction_probabilities']
        
        precision = dict()
        recall = dict()
        avg_precision = dict()
        
        for i in range(n_classes):
            precision[i], recall[i], _ = precision_recall_curve(y_bin[:, i], y_prediction_probabilities[:, i])
            avg_precision[i] = average_precision_score(y_bin[:, i], y_prediction_probabilities[:, i])
            
            ax.plot(recall[i], precision[i], color=colors[i % len(colors)], lw=2,
                    label=f'PR curve of class {target_classes[i]} (AUC-PR = {avg_precision[i]:.2f})')
        
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('Recall')
        ax.set_ylabel('Precision')
        ax.set_title(f'{model_name} Precision-Recall Curve', fontsize=18, fontweight='bold', pad=20)
        ax.legend(loc="lower left", fontsize=10)
        ax.tick_params(axis='both', which='major', labelsize=12)
    
    for ax in axes_flat[num_models:]:
        ax.set_visible(False)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.95, hspace=0.3, wspace=0.3)
    plt.show()
    

def plot_learning_curves_comparison(model_results: dict[str, dict], X: np.ndarray, y: np.ndarray):
    """
    Plots learning curves for multiple classification models in a grid layout.
    
    Args:
    model_results (dict[str, dict]): Dictionary of model results, where each key is a model name
                                     and each value is a dictionary containing the trained model
    X (np.ndarray): Feature matrix
    y (np.ndarray): Target vector
    
    Returns:
    None
    """
    num_models = len(model_results)
    num_rows = math.ceil(num_models / 2)
    fig, axes = plt.subplots(num_rows, 2, figsize=(20, 10*num_rows))
    
    fig.suptitle('Comparison of Learning Curves', fontsize=24, y=1.02)
    
    axes_flat = axes.flatten() if num_models > 1 else [axes]
    
    for ax, (model_name, results) in zip(axes_flat, model_results.items()):
        estimator = results['best_model']
        
        train_sizes, train_scores, test_scores = learning_curve(
            estimator, X, y, cv=5, n_jobs=-1, 
            train_sizes=np.linspace(.1, 1.0, 5))
        
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        
        ax.set_title(f"Learning Curve: {model_name}", fontsize=18, fontweight='bold', pad=20)
        ax.set_xlabel("Training examples")
        ax.set_ylabel("Score")
        ax.fill_between(train_sizes, (train_scores_mean - train_scores_std),
                        (train_scores_mean + train_scores_std), alpha=0.1, color="r")
        ax.fill_between(train_sizes, (test_scores_mean - test_scores_std),
                        (test_scores_mean + test_scores_std), alpha=0.1, color="g")
        ax.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
        ax.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
        ax.legend(loc="best")
        ax.tick_params(axis='both', which='major', labelsize=12)
        
        # Set y-axis limits
        ax.set_ylim(0, 1.1)
    
    for ax in axes_flat[num_models:]:
        ax.set_visible(False)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.95, hspace=0.4, wspace=0.3)
    plt.show()
    

plot_confusion_matrices_comparison(model_results, y_test, target_classes)
plot_roc_curves_comparison(model_results, y_test, target_classes)
plot_precision_recall_curves_comparison(model_results, y_test, target_classes)
plot_learning_curves_comparison(model_results, X, y)

## **Individual Model Performance Visualizations**

### **Plotting Functions**

In [ ]:

def display_model_evaluation_results(model_name: str, results: dict = model_results) -> None:
    """
    Displays the evaluation results for a given model.
    
    Args:
        model_name (str): Name of the model
        results (dict): Dictionary containing the evalutation results of each model
    Returns: None
    """
    display_markdown(md(f"### **Model: {model_name}**"))
    display_markdown(md(f"* #### **F1 Score:** {results[model_name]["f1"]:.{NUM_DECIMAL_PLACES}f}"))
    display_markdown(md(f"* #### **Accuracy:** {results[model_name]["accuracy"]:.{NUM_DECIMAL_PLACES}f}"))
    display_markdown(md(f"* #### **Precision:** {results[model_name]["precision"]:.{NUM_DECIMAL_PLACES}f}"))
    display_markdown(md(f"* #### **Recall:** {results[model_name]["recall"]:.{NUM_DECIMAL_PLACES}f}"))



def plot_confusion_matrix(model_name, y_true, y_predictions, classes = target_classes):
    cm = confusion_matrix(y_true, y_predictions)
    plt.figure(figsize = (12, 8))
    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = classes)
    disp.plot(cmap = "Blues")
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()
    
    
def plot_roc_curve(model_name, y_true, y_prediction_probabilities, classes = target_classes):
    n_classes = len(classes)
    y_bin = label_binarize(y_true, classes = range(n_classes))
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_prediction_probabilities[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    plt.figure(figsize = (10, 8))
    for i, color in zip(range(n_classes), ['blue', 'red', 'green', 'orange']):
        plt.plot(fpr[i], tpr[i], color = color, lw = 2,
                 label = f'ROC curve of class {classes[i]} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw = 2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc = "lower right")
    plt.show()
    
    
def plot_multiclass_precision_recall_curve(model_name, y_true, y_prediction_probabilities, classes = target_classes):
    n_classes = len(classes)
    y_bin = label_binarize(y_true, classes = range(n_classes))
    
    precision = dict()
    recall = dict()
    avg_precision = dict()
    
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_bin[:, i], y_prediction_probabilities[:, i])
        avg_precision[i] = average_precision_score(y_bin[:, i], y_prediction_probabilities[:, i])
    
    plt.figure(figsize=(10, 8))
    for i, color in zip(range(n_classes), ['blue', 'red', 'green', 'orange']):
        plt.plot(recall[i], precision[i], color = color, lw = 2,
                 label = f'Precision-Recall curve of class {classes[i]} (AUC = {avg_precision[i]:.2f})')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'{model_name} Precision-Recall Curve')
    plt.legend(loc = "lower left")
    plt.show()
    

feature_columns = list(X_train.columns)
def get_featuere_names(X_column_names = feature_columns):
    feature_names = [column_name.replace("_", " ") for column_name in X_column_names]
    return feature_names


def plot_feature_importance(model_name, model, feature_names = FEATURE_LIST):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    plt.figure(figsize=(10, 6))
    plt.title(f"{model_name} Feature Importances")
    plt.bar(range(len(importances)), importances[indices])
    plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation = 90)
    plt.tight_layout()
    plt.show()
    

def plot_learning_curve(estimator, X, y, model_name):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, n_jobs=-1, 
        train_sizes = np.linspace(.1, 1.0, 5))
    
    train_scores_mean = np.mean(train_scores, axis = 1)
    train_scores_std = np.std(train_scores, axis = 1)
    test_scores_mean = np.mean(test_scores, axis = 1)
    test_scores_std = np.std(test_scores, axis = 1)
    
    plt.figure(figsize=(10, 6))
    plt.title(f"Learning Curve: {model_name}")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.fill_between(train_sizes, (train_scores_mean - train_scores_std),
                     (train_scores_mean + train_scores_std), alpha = 0.1, color = "r")
    plt.fill_between(train_sizes, (test_scores_mean - test_scores_std),
                     (test_scores_mean + test_scores_std), alpha = 0.1, color = "g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color = "r", label = "Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color = "g", label = "Cross-validation score")
    plt.legend(loc = "best")
    plt.show()

### **Logistic Regression**

In [ ]:
logistic_model_name = "Logistic Regression"

logistic_best_model = model_results[logistic_model_name]["best_model"]

logistic_y_test_predictions = model_results[logistic_model_name]["y_test_predictions"]
logistic_y_test_prediction_probabilities = model_results[logistic_model_name]["y_test_prediction_probabilities"]

logistic_f1_score = model_results[logistic_model_name]["f1"]
logistic_accuracy = model_results[logistic_model_name]["accuracy"]
logistic_precision = model_results[logistic_model_name]["precision"]
logistic_recall = model_results[logistic_model_name]["recall"]

In [ ]:
display_model_evaluation_results(logistic_model_name)
plot_confusion_matrix(logistic_model_name, y_test, logistic_y_test_predictions)
plot_roc_curve(logistic_model_name, y_test, logistic_y_test_prediction_probabilities)
plot_multiclass_precision_recall_curve(logistic_model_name, y_test, logistic_y_test_prediction_probabilities)
plot_learning_curve(logistic_best_model, X_train, y_train, logistic_model_name)

In [ ]:
knn_model_name = "KNN"

knn_best_model = model_results[knn_model_name]["best_model"]

knn_y_test_predictions = model_results[knn_model_name]["y_test_predictions"]
knn_y_test_prediction_probabilities = model_results[knn_model_name]["y_test_prediction_probabilities"]

knn_f1_score = model_results[knn_model_name]["f1"]
knn_accuracy = model_results[knn_model_name]["accuracy"]
knn_precision = model_results[knn_model_name]["precision"]
knn_recall = model_results[knn_model_name]["recall"]

In [ ]:
display_model_evaluation_results(knn_model_name)
plot_confusion_matrix(knn_model_name, y_test, knn_y_test_predictions)
plot_roc_curve(knn_model_name, y_test, knn_y_test_prediction_probabilities)
plot_multiclass_precision_recall_curve(knn_model_name, y_test, knn_y_test_prediction_probabilities)
plot_learning_curve(knn_best_model, X_train, y_train, knn_model_name)

In [ ]:
decision_tree_model_name = "Decision Tree"

decision_tree_best_model = model_results[decision_tree_model_name]["best_model"]

decision_tree_y_test_predictions = model_results[decision_tree_model_name]["y_test_predictions"]
decision_tree_y_test_prediction_probabilities = model_results[decision_tree_model_name]["y_test_prediction_probabilities"]

decision_tree_f1_score = model_results[decision_tree_model_name]["f1"]
decision_tree_accuracy = model_results[decision_tree_model_name]["accuracy"]
decision_tree_precision = model_results[decision_tree_model_name]["precision"]
decision_tree_recall = model_results[decision_tree_model_name]["recall"]

In [ ]:
display_model_evaluation_results(decision_tree_model_name)
plot_confusion_matrix(decision_tree_model_name, y_test, decision_tree_y_test_predictions)
plot_roc_curve(decision_tree_model_name, y_test, decision_tree_y_test_prediction_probabilities)
plot_multiclass_precision_recall_curve(decision_tree_model_name, y_test, decision_tree_y_test_prediction_probabilities)
plot_learning_curve(decision_tree_best_model, X_train, y_train, decision_tree_model_name)

In [ ]:
random_forest_model_name = "Random Forest"

random_forest_best_model = model_results[random_forest_model_name]["best_model"]

random_forest_y_test_predictions = model_results[random_forest_model_name]["y_test_predictions"]
random_forest_y_test_prediction_probabilities = model_results[random_forest_model_name]["y_test_prediction_probabilities"]

random_forest_f1_score = model_results[random_forest_model_name]["f1"]
random_forest_accuracy = model_results[random_forest_model_name]["accuracy"]
random_forest_precision = model_results[random_forest_model_name]["precision"]
random_forest_recall = model_results[random_forest_model_name]["recall"]

In [ ]:
display_model_evaluation_results(random_forest_model_name)
plot_confusion_matrix(random_forest_model_name, y_test, random_forest_y_test_predictions)
plot_roc_curve(random_forest_model_name, y_test, random_forest_y_test_prediction_probabilities)
plot_multiclass_precision_recall_curve(random_forest_model_name, y_test, random_forest_y_test_prediction_probabilities)
plot_learning_curve(random_forest_best_model, X_train, y_train, random_forest_model_name)

In [ ]:
svc_model_name = "SVC"

svc_best_model = model_results[svc_model_name]["best_model"]

svc_y_test_predictions = model_results[svc_model_name]["y_test_predictions"]
svc_y_test_prediction_probabilities = model_results[svc_model_name]["y_test_prediction_probabilities"]

svc_f1_score = model_results[svc_model_name]["f1"]
svc_accuracy = model_results[svc_model_name]["accuracy"]
svc_precision = model_results[svc_model_name]["precision"]
svc_recall = model_results[svc_model_name]["recall"]

In [ ]:
display_model_evaluation_results(svc_model_name)
plot_confusion_matrix(svc_model_name, y_test, svc_y_test_predictions)
plot_roc_curve(svc_model_name, y_test, svc_y_test_prediction_probabilities)
plot_multiclass_precision_recall_curve(svc_model_name, y_test, svc_y_test_prediction_probabilities)
plot_learning_curve(svc_best_model, X_train, y_train, svc_model_name)

In [ ]:
naive_bayes_model_name = "Naive Bayes"

naive_bayes_best_model = model_results[naive_bayes_model_name]["best_model"]

naive_bayes_y_test_predictions = model_results[naive_bayes_model_name]["y_test_predictions"]
naive_bayes_y_test_prediction_probabilities = model_results[naive_bayes_model_name]["y_test_prediction_probabilities"]

naive_bayes_f1_score = model_results[naive_bayes_model_name]["f1"]
naive_bayes_accuracy = model_results[naive_bayes_model_name]["accuracy"]
naive_bayes_precision = model_results[naive_bayes_model_name]["precision"]
naive_bayes_recall = model_results[naive_bayes_model_name]["recall"]

In [ ]:
display_model_evaluation_results(naive_bayes_model_name)
plot_confusion_matrix(naive_bayes_model_name, y_test, naive_bayes_y_test_predictions)
plot_roc_curve(naive_bayes_model_name, y_test, naive_bayes_y_test_prediction_probabilities)
plot_multiclass_precision_recall_curve(naive_bayes_model_name, y_test, naive_bayes_y_test_prediction_probabilities)
plot_learning_curve(naive_bayes_best_model, X_train, y_train, naive_bayes_model_name)